In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
!git clone https://github.com/rafaseto/PIBIC-AnaliseDados.git

Cloning into 'PIBIC-AnaliseDados'...


In [3]:
%cd PIBIC-AnaliseDados

c:\Users\rafae\Documents\somebooks\College\PIBIC\Rafael-AnaliseDados\PIBIC-AnaliseDados


In [4]:
import pip

In [5]:
!pip install dlisio


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from dlisio import dlis
import pandas as pd

In [7]:
# A função 'glob' do módulo 'glob' é usada para procurar todos os arquivos em um diretório com determinada extensão
import glob

In [8]:
dli = []        # Conterá os arquivos lógicos
filenames = []  # Conterá os caminhos dos arquivos dlis
pocos = []      # Conterá strings identificadoras de 9 caracteres, tipo '3-CP-1847'
charId_inicio, charId_fim = 5, 14   # Índices de início e fim da substring identificadora do poço

# Utilizando a função 'glob' do módulo 'glob' para procurar todos os arquivos com extensão DLIS em 'Data'
for file in glob.glob(r'**/Data' + "/*.dlis", recursive=True):
    # Carregando os arquivos dlis com 'load' e armazenando-os em 'leitura'
    # 'tail' recebe valores restantes, caso a função retorne mais de uma peça de informação
    leitura, *tail = dlis.load(f'{file}')
    pocos.append(file[charId_inicio : charId_fim])    # armazenando identificadores do tipo '3-CP-XXXX'
    filenames.append(file)      # armazenando o caminho dos arquivos dlis
    dli.append(leitura)         # armazenando os arquivos lógicos carregados com 'load'

In [9]:
# Criando um dicionário para associar nomes aos elementos de dli
dli_dict = {
    '1847': dli[0],
    '1848': dli[1],
    '1851': dli[2],
    '1853': dli[3],
    '1855': dli[4],
    '1857': dli[5]
}

In [10]:
dli[0].describe()

------------
Logical File
------------
Description : LogicalFile(GEOLOAD.1)
Frames      : 1
Channels    : 21

Known objects
--
FRAME       : 1
TOOL        : 1
NO-FORMAT   : 1
ORIGIN      : 1
COMMENT     : 2
CHANNEL     : 21
PARAMETER   : 167
FILE-HEADER : 1

Unknown objects
--
280-FRAMESTEP-INFO : 1


In [11]:
dli_dict['1847'].describe()

------------
Logical File
------------
Description : LogicalFile(GEOLOAD.1)
Frames      : 1
Channels    : 21

Known objects
--
FRAME       : 1
TOOL        : 1
NO-FORMAT   : 1
ORIGIN      : 1
COMMENT     : 2
CHANNEL     : 21
PARAMETER   : 167
FILE-HEADER : 1

Unknown objects
--
280-FRAMESTEP-INFO : 1


In [12]:
dlis_df = []    # Conterá os dataframes respectivos aos poços

# Curvas de perfis que não serão utilizadas
curvas_nao_utilizadas = ['WF11', 'AHV', 'AHVT', 'BHV', 'BHVT', 'CS', 'FCPS', 'HDCN', 'HMCN', 'ITT', 'ITTT', 'LSPD', 'MMK', 'NCPS', 'SP']

# Iterando sobre os arquivos lógicos de todos os poços (que estão armazenados em 'dli_dict'),
# '.values()' se refere aos valores do dicionário (não às chaves)
for poco in dli_dict.values():
    try:
        # Armazenando as curvas que serão utilizadas em uma lista
        curvas_utilizadas = [
            channel.name                                    # Os elementos da lista serão os nomes das curvas
            for channel in poco.channels                    # As curvas são acessadas por meio de 'poco.channels'
            if channel.name not in curvas_nao_utilizadas    # As curvas que não utilizaremos não serão armazenadas na lista
        ]


        curvas = poco.frames[0].curves()

        # Criando um pandas dataframe do poço respectivo à atual iteração e armazenando o mesmo em dlis_df
        dlis_df.append(pd.DataFrame(curvas[curvas_utilizadas]))
    except:
        pass

In [13]:
# Criando um dicionário para associar nomes aos elementos de dlis_df
dlis_df_dict = {
    '1847' : dlis_df[0],
    '1848' : dlis_df[1],
    '1851' : dlis_df[2],
    '1853' : dlis_df[3],
    '1855' : dlis_df[4],
    '1857' : dlis_df[5]
}

In [14]:
dlis_df_dict['1847']

,TDEP,GR,TENS,BS,NPHI,CALI,RHOB,DRHO,PE,HDRS,HMRS,DTC
0,374.899994,-999.250000,345.178406,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
1,374.799994,-999.250000,346.090210,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
2,374.699994,-999.250000,348.874512,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
3,374.599994,-999.250000,348.379608,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
4,374.499994,-999.250000,349.917908,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
...,...,...,...,...,...,...,...,...,...,...,...,...
3595,15.399989,55.795601,910.639893,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
3596,15.299989,52.518200,910.807007,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
3597,15.199989,48.504501,909.179626,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
3598,15.099989,45.021198,909.942383,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25


In [16]:
from aux_datapreprocess import *

## Remoção de Colunas

In [19]:
colunas_para_remover = ['NPHI', 'CALI', 'RHOB', 'DRHO', 'PE', 'HDRS', 'HMRS', 'DTC']
dlis_df_dict['1847'] = remove_colunas(dlis_df_dict['1847'], colunas_para_remover)

In [21]:
dlis_df_dict['1847']

,profundidade,GR,TENS,BS
0,374.899994,-999.250000,345.178406,8.5
1,374.799994,-999.250000,346.090210,8.5
2,374.699994,-999.250000,348.874512,8.5
3,374.599994,-999.250000,348.379608,8.5
4,374.499994,-999.250000,349.917908,8.5
...,...,...,...,...
3595,15.399989,55.795601,910.639893,8.5
3596,15.299989,52.518200,910.807007,8.5
3597,15.199989,48.504501,909.179626,8.5
3598,15.099989,45.021198,909.942383,8.5


## Renomeação de Colunas

In [26]:
dlis_df_dict['1847'] = renomeiaColuna(dlis_df_dict['1847'], "TDEP", "profundidade")

In [27]:
dlis_df_dict['1847']

,profundidade,GR,TENS,BS
0,374.899994,-999.250000,345.178406,8.5
1,374.799994,-999.250000,346.090210,8.5
2,374.699994,-999.250000,348.874512,8.5
3,374.599994,-999.250000,348.379608,8.5
4,374.499994,-999.250000,349.917908,8.5
...,...,...,...,...
3595,15.399989,55.795601,910.639893,8.5
3596,15.299989,52.518200,910.807007,8.5
3597,15.199989,48.504501,909.179626,8.5
3598,15.099989,45.021198,909.942383,8.5


## Adicionando Labels

In [29]:
add_label(dlis_df, "Poco", [0,1,2,3,4,5], ["P-47", "P-48", "P-51", "P-53", "P-55", "P-57"])

In [30]:
add_label(dlis_df, "XYZ", [0,1,2,3,4,5], ["P-47", "P-48", "P-51", "P-53", "P-55", "P-57"])

In [32]:
dlis_df[0]

,profundidade,GR,TENS,BS,NPHI,CALI,RHOB,DRHO,PE,HDRS,HMRS,DTC,Poco,XYZ
0,374.899994,-999.250000,345.178406,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,P-47,P-47
1,374.799994,-999.250000,346.090210,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,P-47,P-47
2,374.699994,-999.250000,348.874512,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,P-47,P-47
3,374.599994,-999.250000,348.379608,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,P-47,P-47
4,374.499994,-999.250000,349.917908,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,P-47,P-47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,15.399989,55.795601,910.639893,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,P-47,P-47
3596,15.299989,52.518200,910.807007,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,P-47,P-47
3597,15.199989,48.504501,909.179626,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,P-47,P-47
3598,15.099989,45.021198,909.942383,8.5,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,P-47,P-47


In [ ]:
dlis_df[1]

,TDEP,GR,NPHI,RHOB,DRHO,PE,CALI,DTC,HDRS,HMRS,TENS,BSZ,Poco
0,22.1,None,None,None,None,None,None,None,None,None,None,None,P-48
1,22.2,None,None,None,None,None,None,None,None,None,None,None,P-48
2,22.3,None,None,None,None,None,None,None,None,None,None,None,P-48
3,22.4,None,None,None,None,None,None,None,None,None,None,None,P-48
4,22.5,None,None,None,None,None,None,None,None,None,None,None,P-48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,227.9,None,None,None,None,None,None,None,None,None,396.682892,8.5,P-48
2059,228.0,None,None,None,None,None,None,None,None,None,378.842682,8.5,P-48
2060,228.1,None,None,None,None,None,None,None,None,None,370.807678,8.5,P-48
2061,228.2,None,None,None,None,None,None,None,None,None,363.190735,8.5,P-48


## Transformação NPHI

In [ ]:
nphi_transform(dlis_df, [1, 3])

In [ ]:
dlis_df[3]

,TDEP,DTC,HDRS,HMRS,GR,CALI,NPHI,RHOB,DRHO,PE,TENS,BSZ,Poco
0,8.4,None,None,None,None,None,NaN,None,None,None,None,None,P-53
1,8.5,None,None,None,None,None,NaN,None,None,None,None,None,P-53
2,8.6,None,None,None,None,None,NaN,None,None,None,None,None,P-53
3,8.7,None,None,None,None,None,NaN,None,None,None,None,None,P-53
4,8.8,None,None,None,None,None,NaN,None,None,None,None,None,P-53
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3102,318.6,None,None,None,None,None,NaN,None,None,None,1948.277588,8.5,P-53
3103,318.7,None,None,None,None,None,NaN,None,None,None,1798.774292,8.5,P-53
3104,318.8,None,None,None,None,None,NaN,None,None,None,1580.88208,8.5,P-53
3105,318.9,None,None,None,None,None,NaN,None,None,None,1340.839844,8.5,P-53


## Remoção de Pontos com Falha

In [ ]:
pontos_com_falha = [
    (dlis_df[0]['TDEP'] < 250) | (dlis_df[0]['TDEP'] >= 360),
    (dlis_df[1]['TDEP'] <= 75) | (dlis_df[1]['TDEP'] >= 210),
    (dlis_df[2]['TDEP'] <= 60) | (dlis_df[2]['TDEP'] >= 260),
    (dlis_df[3]['TDEP'] <= 90) | (dlis_df[3]['TDEP'] >= 300),
    (dlis_df[4]['TDEP'] <= 64) | (dlis_df[4]['TDEP'] >= 215),
    (dlis_df[5]['TDEP'] <= 90) | (dlis_df[5]['TDEP'] >= 320)
]

# Chamando a função
dataframes_sem_falha = remove_pontos_com_falha(dlis_df, pontos_com_falha)

In [ ]:
dlis_df[3]

,TDEP,DTC,HDRS,HMRS,GR,CALI,NPHI,RHOB,DRHO,PE,TENS,BSZ,Poco
817,90.1,167.007874,8.238415,8.027718,75.416107,8.720348,21.22189,2.501387,0.018591,4.268933,1429.558472,8.5,P-53
818,90.2,151.259842,11.085132,11.028458,62.259724,8.731671,15.034413,2.555811,0.01378,4.671879,1433.620117,8.5,P-53
819,90.3,151.259842,13.338861,13.308826,63.969017,8.732605,14.21414,2.550199,0.018576,4.775152,1429.798218,8.5,P-53
820,90.4,198.503937,12.763572,12.349573,72.722672,8.741086,14.72111,2.525461,0.028277,4.764666,1442.796509,8.5,P-53
821,90.5,198.503937,7.147364,6.389296,95.461456,8.711792,19.147922,2.411309,0.034687,4.497918,1471.055664,8.5,P-53
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,299.5,120.061691,1.373062,1.482344,125.589912,8.684558,32.710385,2.288296,-0.011521,3.895833,1540.631958,8.5,P-53
2912,299.6,120.192993,1.362576,1.455013,127.126549,8.678904,32.588157,2.288607,-0.01674,3.914276,1545.220093,8.5,P-53
2913,299.7,120.551941,1.354665,1.429209,129.819977,8.677963,32.482436,2.290097,-0.015334,3.903882,1554.50415,8.5,P-53
2914,299.8,122.142662,1.344403,1.38702,131.667404,8.681731,33.067477,2.290119,-0.009622,3.880727,1551.772461,8.5,P-53
